In [ ]:
import numpy as np
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import random
from torch.utils.data.sampler import SubsetRandomSampler
import torchvision.transforms as transforms

import os
import shutil
import string
from sklearn.model_selection import train_test_split

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
####################### ATTENTION!!! ##########################
####################### DON'T RUN THIS!!!! #######################

# Shuyan
dataset_path = '/content/gdrive/My Drive/Documents/University Documents/Fourth Year/APS360/APS360 Project/GTSRB_dataset/archive'
# Shihang
# dataset_path =
# Evelyn
# dataset_path =
# Zian
# dataset_path =

# load train_val_set
train_val_path = f'{dataset_path}/Train'

train_val_set = torchvision.datasets.ImageFolder(train_val_path)

print(f'train_val_set size: {len(train_val_set)}')

updated_train_val_path = f'{dataset_path}/Updated_Train'
desired_size = (224, 224)

# limit to 300 images and add zero paddings
for class_id in range(43):
    # make the directory to hold updated data
    os.makedirs(
        os.path.join(updated_train_val_path, str(class_id)),
        exist_ok=True,
    )

    class_folder_path = os.path.join(train_val_path, str(class_id))

    if os.path.exists(class_folder_path):
        all_images = os.listdir(class_folder_path)
        if len(all_images) > 300:
            all_images = all_images[:300]

            print(f'Limited class {class_id} to 300 images')
        else:
            print(f'Class {class_id} has {len(all_images)} images')

        for image in all_images:
            print(f'Processing {image} in class {class_id}')

            image_path = os.path.join(class_folder_path, image)
            updated_class_path = os.path.join(updated_train_val_path, str(class_id))
            updated_image_path = os.path.join(updated_class_path, image)
            with Image.open(image_path) as img:
                # Add padding to the image
                old_size = img.size
                new_size = max(old_size[0], desired_size[0]), max(old_size[1], desired_size[1])
                new_im = Image.new("RGB", new_size, (0, 0, 0))
                new_im.paste(img, ((new_size[0]-old_size[0])//2,
                                  (new_size[1]-old_size[1])//2))

                # save the image:
                new_im.save(updated_image_path)
    else:
        print(f'Class folder {class_id} does not exist')

Streaming output truncated to the last 5000 lines.
Processing 00025_00025_00002.png in class 25
Processing 00025_00002_00020.png in class 25
Processing 00025_00003_00014.png in class 25
Processing 00025_00016_00017.png in class 25
Processing 00025_00009_00020.png in class 25
Processing 00025_00018_00025.png in class 25
Processing 00025_00009_00027.png in class 25
Processing 00025_00012_00009.png in class 25
Processing 00025_00018_00008.png in class 25
Processing 00025_00007_00005.png in class 25
Processing 00025_00027_00019.png in class 25
Processing 00025_00001_00027.png in class 25
Processing 00025_00004_00019.png in class 25
Processing 00025_00020_00018.png in class 25
Processing 00025_00000_00012.png in class 25
Processing 00025_00020_00000.png in class 25
Processing 00025_00025_00023.png in class 25
Processing 00025_00023_00007.png in class 25
Processing 00025_00004_00001.png in class 25
Processing 00025_00004_00004.png in class 25
Processing 00025_00021_00016.png in class 25
Proc

In [ ]:
import os
import pandas as pd
from torch.utils.data import Dataset
from PIL import Image

# Shuyan
dataset_path = '/content/gdrive/My Drive/Documents/University Documents/Fourth Year/APS360/APS360 Project/GTSRB_dataset/archive'
# Shihang
# dataset_path =
# Evelyn
# dataset_path =
# Zian
# dataset_path =

# get train_val_set path
updated_train_val_path = f'{dataset_path}/Updated_Train'

# to test the loading of train_val_set
train_val_set = torchvision.datasets.ImageFolder(updated_train_val_path)
print(f'train_val_set size: {len(train_val_set)}')

# configure test_set
test_path = f'{dataset_path}'
csv_file_path = f'{dataset_path}/Test.csv'
data = pd.read_csv(csv_file_path)

class CustomDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        self.annotations = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir, self.annotations.iloc[idx, -1])
        # print("img_name:", img_name)
        image = Image.open(img_name).convert('RGB')
        label = self.annotations.iloc[idx, -2]
        # print("label:", label)

        if self.transform:
            image = self.transform(image)

        return image, label

# to test the loading of test_set
test_set = CustomDataset(csv_file=csv_file_path,
                        root_dir=test_path)

print(f'test_set size: {len(test_set)}')

# to test the __getitem__ method
img, label = test_set[0]

train_val_set size: 12330
test_set size: 12630


In [ ]:
def get_data_loader(batch_size):
    classes = list(range(0, 43))
    ########################################################################
    # We transform them to Tensors of normalized range [-1, 1].
    transform = transforms.Compose([
        transforms.RandomApply([transforms.RandomRotation(degrees=15)], p=0.5),  # Rotation
        transforms.RandomApply([transforms.RandomAffine(degrees=0, translate=(0.1, 0.1))], p=0.5),  # Translation
        transforms.RandomApply([transforms.ColorJitter(brightness=0.2)], p=0.5),  # Brightness Variation
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)
    )])

    train_val_set = torchvision.datasets.ImageFolder(
        root=updated_train_val_path,
        transform=transform
    )

    print(f'train_val_set size: {len(train_val_set)}')

    # Get the list of indices to sample from
    relevant_train_val_indices = list(range(len(train_val_set)))

    # Split into train and validation
    np.random.seed(1000) # Fixed numpy random seed for reproducible shuffling
    np.random.shuffle(relevant_train_val_indices)
    split = int(len(relevant_train_val_indices) * 0.7) #split at 70%

    # split into training and validation indices
    relevant_train_indices, relevant_val_indices = relevant_train_val_indices[:split], relevant_train_val_indices[split:]
    train_sampler = SubsetRandomSampler(relevant_train_indices)
    train_loader = torch.utils.data.DataLoader(train_val_set, batch_size=batch_size,
                                               num_workers=1, sampler=train_sampler)
    val_sampler = SubsetRandomSampler(relevant_val_indices)
    val_loader = torch.utils.data.DataLoader(train_val_set, batch_size=batch_size,
                                              num_workers=1, sampler=val_sampler)
    # Load testing data
    test_set = CustomDataset(csv_file=csv_file_path,
                             root_dir=test_path,
                             transform=transform)

    # Get the list of indices to sample from
    relevant_test_indices = list(range(len(test_set)))
    test_sampler = SubsetRandomSampler(relevant_test_indices)
    test_loader = torch.utils.data.DataLoader(test_set, batch_size=batch_size,
                                             num_workers=1, sampler=test_sampler)
    print(f'test_set size: {len(test_set)}')
    return train_loader, val_loader, test_loader, classes

In [ ]:
train_loader, val_loader, test_loader, classes = get_data_loader(1)

train_val_set size: 12330
test_set size: 12630
